In [1]:
import ironman
import numpy as np
import pandas as pd
import corner
import matplotlib.pyplot as plt
import rmfit
from astropy import units as u
from astropy import constants as c
import juliet
from astropy.timeseries import LombScargle

In [2]:
#from matplotlib import rc
#rc('font', **{'family':'sans-serif'}) Use only if you have pdflatex (recommended)
#rc('text', usetex=True)

plt.rcParams['xtick.top'] =  True
plt.rcParams['xtick.direction'] =  'in'
plt.rcParams['xtick.major.width'] =  1.0
plt.rcParams['xtick.minor.width'] =  1.0
plt.rcParams['ytick.right'] =  True
plt.rcParams['ytick.direction'] =  'in'
plt.rcParams['ytick.major.width'] = 1.0
plt.rcParams['ytick.minor.width'] =  1.0
plt.rcParams['lines.markeredgewidth'] =  1.0

In [ ]:
dataset = juliet.load(input_folder = 'TOI-2431_juliet_NewDensity_FCO', out_folder = 'TOI-2431_juliet_NewDensity_FCO')
results = dataset.fit(sampler = "dynamic_dynesty")

In [ ]:
#results.nested_sampling.results.logz[-1]

In [ ]:
P = juliet.utils.get_quantiles(results.posteriors['posterior_samples']['P_p1'])[0]
t0 = juliet.utils.get_quantiles(results.posteriors['posterior_samples']['t0_p1'])[0]
print(P,t0)

# Photometry

In [ ]:
def plot_with_gp(instrument, title, name, P, t0):
    global results
    x = dataset.times_lc[instrument]
    phases = juliet.utils.get_phases(dataset.times_lc[instrument], P, t0)
    transit_plus_GP = results.lc.evaluate(instrument)
    transit_model = results.lc.model[instrument]['deterministic']
    gp_model = results.lc.model[instrument]['GP']

    jitter = np.median(results.posteriors['posterior_samples']['sigma_w_'+instrument]) / 1e6
    
    mask = np.logical_or(np.logical_and(phases>-0.05,phases<-0.02),np.logical_and(phases<0.05,phases>0.02))
    #print(mask)
    oot = (dataset.data_lc[instrument] - gp_model)[mask]
    m_inst = 1-np.mean(oot)
    
    # print(x)
    # print(phases)
    # print(transit_plus_GP)
    # print(transit_model)
    # print(gp_model)
    # print(jitter)
    
    #m_inst = juliet.utils.get_quantiles(results.posteriors['posterior_samples']['mflux_'+instrument])[0]
    transit_model = results.lc.model[instrument]['deterministic']
    
    fig, ax = plt.subplots(dpi=100)
    plt.errorbar(phases,dataset.data_lc[instrument]-gp_model+m_inst,yerr = np.sqrt(dataset.errors_lc[instrument]**2.0 + jitter**2.0), fmt = '.', alpha = 0.7)

    idx = np.argsort(phases)
    plt.plot(phases[idx],transit_model[idx], color='black',zorder=10)

    plt.xlim([-0.5,0.5])
    #plt.ylim([0.98,1.02])
    plt.title(title)
    #plt.xlim()
    #plt.ylim([0.995,1.005])
    plt.xlabel('Phase')
    plt.ylabel('Relative flux')
    plt.show()

    return phases, dataset.data_lc[instrument], np.sqrt(dataset.errors_lc[instrument]**2.0 + jitter**2.0), transit_model, m_inst, gp_model

In [ ]:
phase_tess, flux_tess, flux_err_tess, model_tess, m_tess, gp_model_tess = plot_with_gp("TESS","TESS","TESS_det.csv",P,t0)

In [ ]:
def bin_lc(phase, f, num_bins=40):
    # Shift phase from [-0.5, 0.5] to [0, 1] for binning
    phase = np.mod(phase + 1, 1)

    # Define bin edges and centers
    bin_edges = np.linspace(0, 1, num_bins + 1)
    binned_phase = (bin_edges[:-1] + bin_edges[1:]) / 2
    binned_flux = np.full(num_bins, np.nan)  # Initialize with NaNs

    for i in range(num_bins):
        mask = (phase >= bin_edges[i]) & (phase < bin_edges[i + 1])
        if np.any(mask):
            binned_flux[i] = np.mean(f[mask])  # Mean flux in bin

    # Convert phase back to [-0.5, 0.5] for correct plotting
    binned_phase = binned_phase - (binned_phase > 0.5)

    return binned_phase, binned_flux

fig = plt.figure(figsize=(10,6),dpi=150,constrained_layout=True)

gs = fig.add_gridspec(3, 6)
ax1 = fig.add_subplot(gs[:2, :])
ax2 = fig.add_subplot(gs[2, :], sharex = ax1)

phase_bin, flux_bin = bin_lc(phase_tess, flux_tess)

ax1.errorbar(phase_tess*P*24.,flux_tess-gp_model_tess,flux_err_tess,fmt = '.', alpha=0.1,color="cornflowerblue")
idx = np.argsort(phase_tess)
ax1.plot(phase_tess[idx]*P*24.,model_tess[idx], color='crimson',zorder=10,lw=2)

ax1.plot((phase_bin)*P*24.,flux_bin,"o",color="k",zorder=5)

#ax1.set_ylim()
plt.setp(ax1.get_xticklabels(), visible=False)
ax1.set_ylabel('Relative flux',fontsize=14)

res = flux_tess-gp_model_tess-model_tess

phase_res_bin, res_bin = bin_lc(phase_tess, res)

ax2.errorbar(phase_tess*P*24.,res,flux_err_tess,fmt = '.', alpha=0.1,color="cornflowerblue")
ax2.axhline(0.0,lw=2,color="crimson",zorder=30)
ax2.plot((phase_bin)*P*24.,res_bin,"o",color="k",zorder=5)

ax2.set_xlim(-2.5,2.5)
ax2.set_ylim(-0.008,0.008)
#ax2.set_xticks([-2,-1,0,1,2,3])
ax2.set_xlabel(r'Hours from mid-transit',fontsize=14)
ax2.set_ylabel('O - C', fontsize = 14)

ax1.tick_params(labelsize=12)
ax2.tick_params(labelsize=12)

fig.set_facecolor('w')
#plt.tight_layout()
#fig.legend(loc='upper center', bbox_to_anchor=(0.55, 1.07),ncol=6, fancybox=True, fontsize=9)
plt.savefig("TOI-2431_PHOT.png", bbox_inches="tight", dpi=600)
plt.show()
plt.close()

# Radial Velocities

In [ ]:
def plot_rv(inst):

    global results
    global dataset
    
    fig = plt.figure(figsize=(4,4),dpi=150,constrained_layout=True)
    
    min_time, max_time = np.min(dataset.times_rv[inst])-10,\
                         np.max(dataset.times_rv[inst])+10
    
    model_times = np.linspace(min_time,max_time,100000)
    
    keplerian = results.rv.evaluate(inst, t = model_times) - np.median(results.posteriors['posterior_samples']['mu_'+inst])
    
    color = 'cornflowerblue'
    
    # Evaluate the median jitter for the instrument:
    jitter = 0
    # Evaluate the median systemic-velocity:
    mu = np.median(results.posteriors['posterior_samples']['mu_'+inst])

    # Plot original errorbars + jitter (added in quadrature):
    plt.errorbar(dataset.times_rv[inst]-2457000,dataset.data_rv[inst]-mu,\
                 yerr = np.sqrt(dataset.errors_rv[inst]**2+jitter**2),fmt='o',\
                 mec=color, ecolor=color, mfc = 'white', label=inst,\
                 alpha = 0.5, zorder=5)
    
    # Plot Keplerian model:
    plt.plot(model_times-2457000, keplerian,color='black',zorder=1)
    plt.ylabel('RV (m/s)', fontsize = 11)
    plt.xlabel('Time (BJD - 2457000)', fontsize = 11)
    plt.show()

    keplerian = results.rv.evaluate(inst, t = dataset.times_rv[inst]) - np.median(results.posteriors['posterior_samples']['mu_'+inst])

    return dataset.times_rv[inst], dataset.data_rv[inst], np.sqrt(dataset.errors_rv[inst]**2+jitter**2), mu, keplerian

In [ ]:
times_neid1, rv_neid1, rv_err_neid1, gamma_neid1, model_neid1 =  plot_rv("NEID1")
times_neid2, rv_neid2, rv_err_neid2, gamma_neid2, model_neid2 =  plot_rv("NEID2")
times_neid3, rv_neid3, rv_err_neid3, gamma_neid3, model_neid3 =  plot_rv("NEID3")

In [ ]:
min_time, max_time = np.min(dataset.times_rv["NEID1"]),np.max(dataset.times_rv["NEID3"])
model_times = np.linspace(min_time-10,max_time+10,5000)    
keplerian = results.rv.evaluate("NEID1", t = model_times, nsamples = 5000, return_samples = True)[0] - np.median(results.posteriors['posterior_samples']['mu_NEID1'])

In [ ]:
def keplerian_eval(times, num_samples):
    """
    
    times: Times we want the RV Curve to be evaluated at.
    num_samples: Number of RV Curves we want to sample to model our significance intervals.
    
    """

    #We first call the entire chain from our Juliet run.
    chain = results.posteriors["posterior_samples"]

    #We also define a list to store our RV model evaluation samples.
    #We basically want to have an array of different models so that we can create the shaded areas for sigmas.
    rv_models = []

    #We also define the length of the chain as follows.
    #Note that we chose one parameter to get the amount of posteriors we have in a chain per parameter.
    chain_len = len(chain["K_p1"])
    
    #We then sample from the chain randomly per number of samples user has provided.
    for i in range(num_samples):
        #We get a random index.
        #For this, we sample a random number from 0 to number of chains.
        idx = np.random.randint(0, chain_len)
        
        #We first sample randomly from the posterior chains of K, P and T0.
        #Note that eccentricity and argument of periapsis were kept fixed in the juliet run.
        K = chain["K_p1"][idx]
        P = chain["P_p1"][idx]
        t0 = chain["t0_p1"][idx]
        e = 0
        w = 90

        #We then get the RV Model as follows. (Note: This is without the jitter term, so it is normalized to zero.)
        rv_model = rmfit.get_rv_curve(times, P, t0, e, w, K, plot=False, verbose=False)

        #We add the rv_model into our rv_models list.
        rv_models.append(rv_model)

    return rv_models

In [ ]:
#We then evaluate our keplerian model using the evaluation function.
keplerian = keplerian_eval(model_times, 5000)

In [ ]:
fig = plt.figure(figsize=(10,6),dpi=150,constrained_layout=True)

gs = fig.add_gridspec(3, 6)
ax1 = fig.add_subplot(gs[:2, :])
ax2 = fig.add_subplot(gs[2, :], sharex = ax1)

phase_model = (((model_times-t0 + 0.5*P) % P)/P) - 0.5
phase_neid1 = (((times_neid1-t0 + 0.5*P) % P)/P) - 0.5
phase_neid2 = (((times_neid2-t0 + 0.5*P) % P)/P) - 0.5
phase_neid3 = (((times_neid3-t0 + 0.5*P) % P)/P) - 0.5

plt.setp(ax1.get_xticklabels(), visible=False)
ax1.set_ylabel('RV (m/s)',fontsize=14)

ax1.errorbar(phase_neid1,rv_neid1-gamma_neid1,rv_err_neid1,fmt="o",color="k",capsize=2,elinewidth=1,markerfacecolor="cornflowerblue",label="NEID 1")
ax1.errorbar(phase_neid2,rv_neid2-gamma_neid2,rv_err_neid2,fmt="o",color="k",capsize=2,elinewidth=1,markerfacecolor="yellow",label="NEID 2")
ax1.errorbar(phase_neid3,rv_neid3-gamma_neid3,rv_err_neid3,fmt="o",color="k",capsize=2,elinewidth=1,markerfacecolor="limegreen",label="NEID 3")
idx = np.argsort(phase_model)
ax1.plot(phase_model[idx],np.quantile(keplerian,0.5,axis=0)[idx],lw=2,color="crimson",zorder=20)
ax1.fill_between(phase_model[idx],np.quantile(keplerian,0.16,axis=0)[idx],np.quantile(keplerian,0.84,axis=0)[idx],alpha=0.3,color="#EE2C2C",lw=0,zorder=-1)
ax1.fill_between(phase_model[idx],np.quantile(keplerian,0.02,axis=0)[idx],np.quantile(keplerian,0.98,axis=0)[idx],alpha=0.2,color="#EE2C2C",lw=0,zorder=-1)
ax1.fill_between(phase_model[idx],np.quantile(keplerian,0.0015,axis=0)[idx],np.quantile(keplerian,0.9985,axis=0)[idx],alpha=0.1,color="#EE2C2C",lw=0,zorder=-1)

res_neid1 = rv_neid1-gamma_neid1-model_neid1
res_neid2 = rv_neid2-gamma_neid2-model_neid2
res_neid3 = rv_neid3-gamma_neid3-model_neid3

ax2.errorbar(phase_neid1,res_neid1,rv_err_neid1,fmt="o",color="k",capsize=2,elinewidth=1,markerfacecolor="cornflowerblue")
ax2.errorbar(phase_neid2,res_neid2,rv_err_neid2,fmt="o",color="k",capsize=2,elinewidth=1,markerfacecolor="yellow")
ax2.errorbar(phase_neid3,res_neid3,rv_err_neid3,fmt="o",color="k",capsize=2,elinewidth=1,markerfacecolor="limegreen")
ax2.axhline(0.0,lw=2,color="crimson",zorder=30)

ax2.set_xlim(-0.5,0.5)
ax2.set_ylim(-40,40)
ax2.set_yticks([-25,0,25])
ax2.set_xlabel(r'Phase',fontsize=14)
ax2.set_ylabel('O - C', fontsize = 14)

ax1.tick_params(labelsize=12)
ax2.tick_params(labelsize=12)

fig.set_facecolor('w')
#plt.tight_layout()
fig.legend(loc='upper center', bbox_to_anchor=(0.57, 1.07),ncol=6, fancybox=True, fontsize=9)
plt.savefig("TOI-2431_RVs_phase.png", bbox_inches="tight", dpi=600)
plt.show()
plt.close()

In [ ]:
def get_vals(vec):
    fvec   = np.sort(vec)

    fval  = np.median(fvec)
    nn = int(np.around(len(fvec)*0.15865))

    vali,valf = fval - fvec[nn],fvec[-nn] - fval
    return fval,vali,valf
    
def print_mass_radius_rho_sma_planet(rstar=1.0, rstar_err=0.1, mstar=1.0, mstar_err=0.1, r_units=u.Rjup, sma_units=u.AU, m_units=u.Mjup):
    """
    Calculate and print the planet's mass, radius, density, and semi-major axis. If r_star and/or m_star are not sampled they must be added as inputs

    Parameters:
    rstar (float) optional: Mean stellar radius in solar radii. Only if r_star not sampled
    rstar_err (float) optional: Error in stellar radius. Only if r_star not sampled
    mstar (float) optional: Mean stellar mass in solar masses. Only if m_star not sampled
    mstar_err (float) optional: Error in stellar mass. Only if m_star not sampled
    r_units (Astropy Unit): Desired output units for the planet's radius.
    sma_units (Astropy Unit): Desired output units for the semi-major axis.
    m_units (Astropy Unit): Desired output units for the planet's mass.
    """
    # chain = self.fit.chain
    Rs = np.random.normal(rstar, rstar_err, len(results.posteriors['posterior_samples']['K_p1'])) * u.Rsun
    Ms = np.random.normal(mstar, mstar_err, len(results.posteriors['posterior_samples']['K_p1'])) * u.Msun

    rp = results.posteriors['posterior_samples']['p_p1'] * Rs
    rp = rp.to(r_units)
    r_planet_val, r_planet_down, r_planet_up = get_vals(rp.value)
    print(f"R_planet: {r_planet_val:.4f} +{r_planet_up:.4f} -{r_planet_down:.4f} {r_units}")

    transit_depth = (rp / Rs.to(r_units))**2
    transit_depth_val, transit_depth_down, transit_depth_up = get_vals(transit_depth.value)
    print(f"Transit depth: {transit_depth_val:.6f} +{transit_depth_up:.6f} -{transit_depth_down:.6f}")
    
    P_sec = results.posteriors['posterior_samples']['P_p1'] * u.day
    P_sec = P_sec.to(u.s)
    aRs = results.posteriors['posterior_samples']['a_p1'] * Rs
    sma = aRs.to(sma_units)
    sma_val, sma_down, sma_up = get_vals(sma.value)
    print(f"Sma_planet: {sma_val:.4f} +{sma_up:.4f} -{sma_down:.4f} {sma_units}")

    K = results.posteriors['posterior_samples']['K_p1'] * u.m / u.s
    e = 0
    cos_inc = (results.posteriors['posterior_samples']['b_p1'] * Rs) / sma
    cos_inc = np.clip(cos_inc, -1, 1)
    inc = np.arccos(cos_inc)
    inc_val, inc_down, inc_up = get_vals(inc.value)
    print(f"Inclination: {np.degrees(inc_val):.4f} +{np.degrees(inc_up):.4f} -{np.degrees(inc_down):.4f} deg")
    #inc = 73.83677068878853 * np.pi / 180.0  # Convert to radians
    mp = K * np.sqrt(Ms * sma * (1 - e**2) / c.G) / np.sin(inc)
    mp = mp.to(m_units)
    m_planet_val, m_planet_down, m_planet_up = get_vals(mp.value)
    print(f"M_planet: {m_planet_val:.4f} +{m_planet_up:.4f} -{m_planet_down:.4f} {m_units}")

    dens = (3 * mp / (4 * np.pi * rp**3)).to(u.g / u.cm**3)
    dens_val, dens_down, dens_up = get_vals(dens.value)
    print(f"Rho_planet: {dens_val:.4f} +{dens_up:.4f} -{dens_down:.4f} g/cm^3")

In [ ]:
#Stellar Density calculation
def rho_star_from_P_and_aRs(P,aRs,return_cgs=True):
    """
    Density of the star from period of planet P, and aRs from transit observations.
    
    INPUT:
    - P in days
    - aRs - unitless

    OUTPUT:
    density of the star
    FLAG: return_cgs:
    -- if true: return in g/cm^3
    -- else: return in solar densities
    
    NOTE:
    - Assumes that eccentricity of the planet is 0. Otherwise not valid
    - Uses equation from Seager et al. 2003
    
    EXAMPLE:
        rho_star_from_P_and_aRs(0.2803,2.3,return_cgs=True) # Should be 2.92
        
        P = np.random.normal(0.2803226,0.0000013,size=1000)
        a = np.random.normal(2.31,0.08,size=1000)
        dens = rho_star_from_P_and_aRs(P,a,return_cgs=True)
        df = pd.DataFrame(dens)
        astropylib.mcFunc.calc_medvals2(df)
    """
    per = P*24.*60.*60 # Change to seconds
    rho_sun = c.M_sun.value/(4.*np.pi*(c.R_sun.value**3.)/3.) # kg/m^3

    # Density from transit, see equation in Seager et al. 2003
    rho_star = 3.*np.pi*(aRs**3.)/(c.G.value*(per**2.)) # kg/m^3
    if return_cgs:
        return rho_star * (1000./(100.*100.*100.)) # g/cm^3
    else:
        return rho_star/rho_sun

In [ ]:
period_result = get_vals(results.posteriors['posterior_samples']['P_p1'])
a_result = get_vals(results.posteriors['posterior_samples']['a_p1'])
P_vec = np.random.normal(period_result[0], period_result[1], 1000)
a_vec = np.random.normal(a_result[0], a_result[1], 1000)

rho_star = rho_star_from_P_and_aRs(P_vec, a_vec, return_cgs = True)
rho_star = get_vals(rho_star)

In [ ]:
print_mass_radius_rho_sma_planet(rstar=0.66,rstar_err=0.01,mstar=0.66,mstar_err=0.02,r_units=u.Rearth,m_units=u.Mearth)
print(f"Rho_star: {rho_star[0]:.4f} +{rho_star[2]:.4f} -{rho_star[1]:.4f} g/cm^3")

In [ ]:
from matplotlib.lines import Line2D

fig, axes = plt.subplots(2, 2, figsize=(12, 6), dpi=150, gridspec_kw={'height_ratios': [2, 1]}, sharex='col')

ax1, ax2 = axes[:, 0]  #Left column (Transit plot)
ax3, ax4 = axes[:, 1]  #Right column (RV plot)

#--------- LEFT PANEL: TRANSIT LIGHT CURVE ---------
phase_bin, flux_bin = bin_lc(phase_tess, flux_tess)

ax1.errorbar(phase_tess * P * 24., flux_tess - gp_model_tess, flux_err_tess, fmt='.', alpha=0.1, color="cornflowerblue", label = "TESS Data")
idx = np.argsort(phase_tess)
ax1.plot(phase_tess[idx] * P * 24., model_tess[idx], color='crimson', zorder=10, lw=2)
ax1.plot((phase_bin) * P * 24., flux_bin, "o", color="k", zorder=5, label = "Binned TESS data")
ax1.set_ylim(0.996, 1.005)
ax1.set_ylabel('Relative flux', fontsize=14)
plt.setp(ax1.get_xticklabels(), visible=False)

res = flux_tess - gp_model_tess - model_tess
phase_res_bin, res_bin = bin_lc(phase_tess, res)

ax2.errorbar(phase_tess * P * 24., res, flux_err_tess, fmt='.', alpha=0.1, color="cornflowerblue")
ax2.axhline(0.0, lw=2, color="crimson", zorder=30)
ax2.plot((phase_bin) * P * 24., res_bin, "o", color="k", zorder=5)
ax2.set_xlim(-2.5, 2.5)
ax2.set_ylim(-0.004, 0.004)
ax2.set_xlabel(r'Hours from mid-transit', fontsize=14)
ax2.set_ylabel('O - C', fontsize=14)

#ax1.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3, fancybox=True, fontsize=9)
ax1.legend(
    handles=[
        Line2D([0], [0], marker='o', color='cornflowerblue', markersize=2, linestyle='None', label="TESS Data", alpha=1.0),
        Line2D([0], [0], marker='o', color='k', markersize=6, linestyle='None', label="Binned TESS data")
    ],
    loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3, fancybox=True, fontsize=9
)

# Label "a)"
ax1.text(-2.4, 1.004, "a)", fontsize=16, fontweight='bold')

#--------- RIGHT PANEL: RADIAL VELOCITY CURVE ---------
phase_model = (((model_times - t0 + 0.5 * P) % P) / P) - 0.5
phase_neid1 = (((times_neid1 - t0 + 0.5 * P) % P) / P) - 0.5
phase_neid2 = (((times_neid2 - t0 + 0.5 * P) % P) / P) - 0.5
phase_neid3 = (((times_neid3 - t0 + 0.5 * P) % P) / P) - 0.5

ax3.errorbar(phase_neid1, rv_neid1 - gamma_neid1, rv_err_neid1, fmt="o", color="k", capsize=2, elinewidth=1, markerfacecolor="cornflowerblue", label="NEID 1")
ax3.errorbar(phase_neid2, rv_neid2 - gamma_neid2, rv_err_neid2, fmt="o", color="k", capsize=2, elinewidth=1, markerfacecolor="yellow", label="NEID 2")
ax3.errorbar(phase_neid3, rv_neid3 - gamma_neid3, rv_err_neid3, fmt="o", color="k", capsize=2, elinewidth=1, markerfacecolor="limegreen", label="NEID 3")

idx = np.argsort(phase_model)
ax3.plot(phase_model[idx], np.quantile(keplerian, 0.5, axis=0)[idx], lw=2, color="crimson", zorder=20)
ax3.fill_between(phase_model[idx], np.quantile(keplerian, 0.16, axis=0)[idx], np.quantile(keplerian, 0.84, axis=0)[idx], alpha=0.3, color="#EE2C2C", lw=0, zorder=-1)
ax3.fill_between(phase_model[idx], np.quantile(keplerian, 0.02, axis=0)[idx], np.quantile(keplerian, 0.98, axis=0)[idx], alpha=0.2, color="#EE2C2C", lw=0, zorder=-1)
ax3.fill_between(phase_model[idx], np.quantile(keplerian, 0.0015, axis=0)[idx], np.quantile(keplerian, 0.9985, axis=0)[idx], alpha=0.1, color="#EE2C2C", lw=0, zorder=-1)
ax3.axhline(0.0, lw=1, ls = "--", color = "grey", zorder = -1000)

ax3.set_ylabel('RV (m/s)', fontsize=14)
plt.setp(ax3.get_xticklabels(), visible=False)

res_neid1 = rv_neid1 - gamma_neid1 - model_neid1
res_neid2 = rv_neid2 - gamma_neid2 - model_neid2
res_neid3 = rv_neid3 - gamma_neid3 - model_neid3

ax4.errorbar(phase_neid1, res_neid1, rv_err_neid1, fmt="o", color="k", capsize=2, elinewidth=1, markerfacecolor="cornflowerblue")
ax4.errorbar(phase_neid2, res_neid2, rv_err_neid2, fmt="o", color="k", capsize=2, elinewidth=1, markerfacecolor="yellow")
ax4.errorbar(phase_neid3, res_neid3, rv_err_neid3, fmt="o", color="k", capsize=2, elinewidth=1, markerfacecolor="limegreen")
ax4.axhline(0.0, lw=2, color="crimson", zorder=30)

ax4.set_xlim(-0.5, 0.5)
ax4.set_ylim(-10, 10)
ax4.set_yticks([-5, 0, 5])
ax4.set_xlabel(r'Phase', fontsize=14)
ax4.set_ylabel('O - C', fontsize=14)

ax3.text(-0.48, 12, "b)", fontsize=16, fontweight='bold')

fig.set_facecolor('w')
ax3.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3, fancybox=True, fontsize=9)

plt.savefig("TOI-2431_Transit_RV.png", bbox_inches="tight", dpi=600)
plt.show()
plt.close()

In [ ]:
from matplotlib.lines import Line2D

fig, axes = plt.subplots(2, 2, figsize=(12, 6), dpi=150, gridspec_kw={'height_ratios': [2, 1]}, sharex='col')

ax1, ax2 = axes[:, 0]  #Left column (Transit plot)
ax3, ax4 = axes[:, 1]  #Right column (RV plot)

#--------- LEFT PANEL: TRANSIT LIGHT CURVE ---------
phase_bin, flux_bin = bin_lc(phase_tess, flux_tess)

ax1.errorbar(phase_tess * P * 24., flux_tess - gp_model_tess, flux_err_tess, fmt='.', alpha=0.1, color="cornflowerblue", label = "TESS Data")
idx = np.argsort(phase_tess)
ax1.plot(phase_tess[idx] * P * 24., model_tess[idx], color='crimson', zorder=10, lw=2)
ax1.plot((phase_bin) * P * 24., flux_bin, "o", color="k", zorder=5, label = "Binned TESS data")
ax1.set_ylim(0.996, 1.005)
ax1.set_ylabel('Relative flux', fontsize=14)
plt.setp(ax1.get_xticklabels(), visible=False)

res = flux_tess - gp_model_tess - model_tess
phase_res_bin, res_bin = bin_lc(phase_tess, res)

ax2.errorbar(phase_tess * P * 24., res, flux_err_tess, fmt='.', alpha=0.1, color="cornflowerblue")
ax2.axhline(0.0, lw=2, color="crimson", zorder=30)
ax2.plot((phase_bin) * P * 24., res_bin, "o", color="k", zorder=5)
ax2.set_xlim(-2.5, 2.5)
ax2.set_ylim(-0.004, 0.004)
ax2.set_xlabel(r'Hours from mid-transit', fontsize=14)
ax2.set_ylabel('O - C', fontsize=14)

#ax1.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3, fancybox=True, fontsize=9)
ax1.legend(
    handles=[
        Line2D([0], [0], marker='o', color='cornflowerblue', markersize=2, linestyle='None', label="TESS Data", alpha=1.0),
        Line2D([0], [0], marker='o', color='k', markersize=6, linestyle='None', label="Binned TESS data")
    ],
    loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3, fancybox=True, fontsize=9
)

# Label "a)"
ax1.text(-2.4, 1.004, "a)", fontsize=16, fontweight='bold')

#--------- RIGHT PANEL: RADIAL VELOCITY CURVE ---------
phase_model = (((model_times - t0 + 0.5 * P) % P) / P) - 0.5
phase_neid1 = (((times_neid1 - t0 + 0.5 * P) % P) / P) - 0.5
phase_neid2 = (((times_neid2 - t0 + 0.5 * P) % P) / P) - 0.5
phase_neid3 = (((times_neid3 - t0 + 0.5 * P) % P) / P) - 0.5

ax3.errorbar(phase_neid1, rv_neid1 - gamma_neid1, rv_err_neid1, fmt = 'o', mec='black', mfc = "cornflowerblue", \
             ecolor = "black", elinewidth=1, zorder = 10000, label="NEID 1")
ax3.errorbar(phase_neid2, rv_neid2 - gamma_neid2, rv_err_neid2, fmt = 'o', mec='black', mfc = "yellow", \
             ecolor = "black", elinewidth=1, zorder = 10000, label="NEID 2")
ax3.errorbar(phase_neid3, rv_neid3 - gamma_neid3, rv_err_neid3, fmt = 'o', mec='black', mfc = "limegreen", \
             ecolor = "black", elinewidth=1, zorder = 10000, label="NEID 3")

idx = np.argsort(phase_model)
ax3.plot(phase_model[idx], np.quantile(keplerian, 0.5, axis=0)[idx], lw=2, color="crimson", zorder=20)
ax3.fill_between(phase_model[idx], np.quantile(keplerian, 0.16, axis=0)[idx], np.quantile(keplerian, 0.84, axis=0)[idx], alpha=0.3, color="#EE2C2C", lw=0, zorder=-1)
ax3.fill_between(phase_model[idx], np.quantile(keplerian, 0.02, axis=0)[idx], np.quantile(keplerian, 0.98, axis=0)[idx], alpha=0.2, color="#EE2C2C", lw=0, zorder=-1)
ax3.fill_between(phase_model[idx], np.quantile(keplerian, 0.0015, axis=0)[idx], np.quantile(keplerian, 0.9985, axis=0)[idx], alpha=0.1, color="#EE2C2C", lw=0, zorder=-1)
ax3.axhline(0.0, lw=1, ls = "--", color = "grey", zorder = -1000)

ax3.set_ylabel('RV (m/s)', fontsize=14)
plt.setp(ax3.get_xticklabels(), visible=False)

res_neid1 = rv_neid1 - gamma_neid1 - model_neid1
res_neid2 = rv_neid2 - gamma_neid2 - model_neid2
res_neid3 = rv_neid3 - gamma_neid3 - model_neid3

ax4.errorbar(phase_neid1, res_neid1, rv_err_neid1, fmt = 'o', mec='black', mfc = "cornflowerblue", \
             ecolor = "black", elinewidth=1, zorder = 10000)
ax4.errorbar(phase_neid2, res_neid2, rv_err_neid2, fmt = 'o', mec='black', mfc = "yellow", \
             ecolor = "black", elinewidth=1, zorder = 10000)
ax4.errorbar(phase_neid3, res_neid3, rv_err_neid3, fmt = 'o', mec='black', mfc = "limegreen", \
             ecolor = "black", elinewidth=1, zorder = 10000)
ax4.axhline(0.0, lw=2, color="crimson", zorder=30)

ax4.set_xlim(-0.5, 0.5)
ax4.set_ylim(-10, 10)
ax4.set_yticks([-5, 0, 5])
ax4.set_xlabel(r'Phase', fontsize=14)
ax4.set_ylabel('O - C', fontsize=14)

ax3.text(-0.48, 12, "b)", fontsize=16, fontweight='bold')

fig.set_facecolor('w')
ax3.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3, fancybox=True, fontsize=9)

plt.savefig("TOI-2431_Transit_RV.png", bbox_inches="tight", dpi=600)
plt.show()
plt.close()